In [1]:
# Enhenced by unsubscribes 20191205

import pandas as pd
import numpy as np
import datetime
import os
import glob
import gc

print(datetime.datetime.now())


def recursive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)
            
os.getcwd()

track_start_date="2018-09-01"

writer_folder="/home/jian/Projects/Big_Lots/Analysis/2019_Q4/LTV_retaintion_rate/output/output_"+str(datetime.datetime.now().date())+"/"
try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)

2019-12-05 11:49:03.834158


In [2]:
# July 1 2018 to Oct 2019

list_weekly_signs_files=list(recursive_file_gen("/home/jian/BigLots/"))
list_weekly_signs_files=[x for x in list_weekly_signs_files if "master" in x.lower()]
list_weekly_signs_files=[x for x in list_weekly_signs_files if x[-4:]==".txt"]
list_weekly_signs_files=[x for x in list_weekly_signs_files if "MediaStorm_" in x]
list_weekly_signs_files=[x for x in list_weekly_signs_files if "weekly" in x.lower()]

list_weekly_signs_files.sort()

In [3]:
df_new_signs_since_Sep18=pd.DataFrame()

for file in list_weekly_signs_files:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','sign_up_date'])
    df=df[df['sign_up_date']>=track_start_date]
    df_new_signs_since_Sep18=df_new_signs_since_Sep18.append(df)

In [4]:
dfdetail_1=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/All Rewards Members 2018-02-04 - 2019-05-04.zip",
                         dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','sign_up_date'])
dfdetail_2=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/MediaStorm Rewards Master P4 2019 - no transaction info.zip",
                         dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','sign_up_date'])

dfdetail_1=dfdetail_1[dfdetail_1['sign_up_date']>=track_start_date]
dfdetail_2=dfdetail_2[dfdetail_2['sign_up_date']>=track_start_date]

df_new_signs_since_Sep18=df_new_signs_since_Sep18.append(dfdetail_1).append(dfdetail_2)

del dfdetail_1
del dfdetail_2

gc.collect()

0

In [5]:
print(df_new_signs_since_Sep18.shape)
df_new_signs_since_Sep18=df_new_signs_since_Sep18.sort_values("sign_up_date",ascending=False).drop_duplicates("customer_id_hashed")
print(df_new_signs_since_Sep18.shape)

(12142496, 3)
(8763575, 3)


In [6]:
month_date_count=df_new_signs_since_Sep18[['sign_up_date']].drop_duplicates()
month_date_count['registerd_month']=month_date_count['sign_up_date'].apply(lambda x: x[:7])
month_date_count=month_date_count.groupby("registerd_month")['sign_up_date'].nunique().to_frame().reset_index()

In [7]:
df_output_long=df_new_signs_since_Sep18[['customer_id_hashed','sign_up_date']]
df_output_long['registerd_month']=df_output_long['sign_up_date'].apply(lambda x: x[:7])
del df_new_signs_since_Sep18

list_ids_new_signs_since_Sep18=df_output_long['customer_id_hashed'].tolist()
gc.collect()

print(df_output_long.shape)
df_output_long.head(3)

(8763575, 3)


,customer_id_hashed,sign_up_date,registerd_month
201365,14d2626ab97b1895f5c0955d635f2d859d6b7773d24a18...,2019-11-30,2019-11
1264,0d486acc9776c5daad35c2bde37be6d25e59ae04681585...,2019-11-30,2019-11
1267,ac101138924bb47feb91fb5ce10178c509ac5830bc9a87...,2019-11-30,2019-11


In [8]:
min_month_str=df_output_long['registerd_month'].min()
min_date_str=df_output_long['sign_up_date'].min()
print("min_signup: ",min_date_str,min_month_str)
print("max_signup: ",df_output_long['sign_up_date'].max(),df_output_long['registerd_month'].max())

min_signup:  2018-09-01 2018-09
max_signup:  2019-11-30 2019-11


In [9]:
# Sales all since 2018-09-01

list_hist_item=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")
list_hist_item=[x for x in list_hist_item if x.split("MediaStormDailySalesHistory")[1][:8]>=track_start_date.replace("-","")]
list_hist_item.sort()
last_weekend_dt_historical=max([x.split("MediaStormDailySalesHistory")[1][:8] for x in list_hist_item])
print(last_weekend_dt_historical[:4]+"-"+last_weekend_dt_historical[4:6]+"-"+last_weekend_dt_historical[6:8])

2019-02-09


In [10]:
list_weekly_item=list(recursive_file_gen("/home/jian/BigLots/"))
list_weekly_item=[x for x in list_weekly_item if "daily" in x.lower() and x[-4:]==".txt"]
list_weekly_item=[x for x in list_weekly_item if "/MediaStorm_" in x]
list_weekly_item.sort()
list_weekly_item=[x for x in list_weekly_item if x.split("/MediaStorm_")[1][:10]>last_weekend_dt_historical[:4]+"-"+last_weekend_dt_historical[4:6]+"-"+last_weekend_dt_historical[6:8]]
# list_weekly_item


list_all_POS_item=list_hist_item+list_weekly_item
print(len(list_all_POS_item))

66


In [11]:
pd.read_table(file,dtype=str,sep="|",nrows=10)

,customer_id_hashed,email_address_hash,sign_up_date,sign_up_channel,sign_up_location,customer_zip_code,transaction_count,transaction_amount,experian_multi_cluster,experian_demo_cluster,purchase_channel,email_unsubscribe_indicator,email_undeliverable_indicator
0,067116941a819b0641bf61a5da4ed180d00d0f3b7c16d6...,ea0585343887acfea493e2eb7e9c8ae099115621785703...,2019-11-25,STORE,296,41011,000000000000001.,0000000000000141.70,0,0,STORES,N,N
1,a66303919c7b02a36eebfce6f4e696c07f689131cab1aa...,5a5d801f597bd4bf0db773e7c05449e71269d41bf4078c...,2019-11-30,STORE,5353,16701,000000000000001.,0000000000000007.40,0,0,STORES,N,N
2,e9f2993a8d49cbf2ec429149c34adecee8fba0d1942ddd...,d16a29498bf3327a6362f970cc20310b27e04aa6eecb6a...,2019-11-27,STORE,4690,54555,000000000000001.,0000000000000143.99,0,0,STORES,N,N
3,4711a6e9ce5cf8269737b20a68e4949d6c4dac0222fd0e...,a67e2f5cb829bf69d2507416aca0275a0822f090b85d81...,2019-11-26,ONLINE,6990,NaN,000000000000000.,0000000000000000.00,0,0,NONE,N,N
4,0142304a625feb56c3e26303c9f2079195f322b23f43e3...,3659add4d23d9bec2cabb6a645badf5122594f8430985d...,2019-11-27,STORE,1877,25177,000000000000001.,0000000000000044.08,0,0,STORES,N,N
5,4c32a03431e4f80cc1a90920e9c2f8305267305c833057...,d2e3c9e4845ce00cc492c92ca7c5e122c2650c82134b2b...,2019-11-24,ONLINE,0,NaN,000000000000000.,0000000000000000.00,0,0,NONE,N,N
6,677ba9782fa14c71089601cc554801919058272bf8edd9...,d6939892de840c5f6cd3d58dedbdffddc97181dffb0dc3...,2019-11-25,STORE,1520,14217,000000000000001.,0000000000000599.98,0,0,STORES,N,N
7,f09c14a7b0a31a776af8363500b45ca08e009e98303f16...,554440301560f357bb07ea456e1cd350069e732e3fe95a...,2019-11-24,STORE,5205,14471,000000000000002.,0000000000000100.62,0,0,STORES,N,N
8,3c4431007ad37f2eb7ab2a7d48b41b7b5604557debf1eb...,57ecd782e85023b5bd55112f762004b3b7b6c0018cb8b6...,2019-11-28,STORE,1850,35810,000000000000001.,0000000000000470.64,0,0,STORES,N,N
9,44d8d200801e9395684fa6cdd164a49332d1070f6fff46...,d347bab561eae38345f5573be5161606a871eddae63970...,2019-11-30,STORE,225,24251,000000000000001.,0000000000000106.15,0,0,STORES,N,N


In [12]:
mapping_division_class=pd.read_table("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20191101-134011-956.txt",
                                    dtype=str,sep="|",usecols=['division_id','class_code_id']).drop_duplicates()
print(mapping_division_class.shape)
mapping_division_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",sep="|",dtype=str)
mapping_division_class=pd.merge(mapping_division_class,mapping_division_name,on="division_id",how="left")
print(mapping_division_class.shape)

(463, 2)
(463, 3)


In [13]:
shopper_by_day_department_list=[]
shopper_by_day_list=[]
i_counter=0
for file in list_all_POS_item:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','class_code_id','item_transaction_amt'])
    df=df[pd.notnull(df['customer_id_hashed'])]
    df=df[df['customer_id_hashed'].isin(list_ids_new_signs_since_Sep18)]
    if i_counter==0:
        df=df[df['transaction_dt']>=track_start_date]
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    #
    df_sales_depart=df.groupby(['customer_id_hashed','transaction_dt','class_code_id'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"sales"})
    shopper_by_day_department_list.append(df_sales_depart)
    
    #
    df_sales=df.groupby(['customer_id_hashed','transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"sales"})
    #
    df=df[df['item_transaction_amt']>0]

    df_trans=df[['customer_id_hashed','transaction_dt','transaction_id','location_id']].drop_duplicates()
    df_trans=df_trans.groupby(['customer_id_hashed','transaction_dt'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
    
    df=pd.merge(df_sales,df_trans,on=['customer_id_hashed','transaction_dt'],how="outer")
    shopper_by_day_list.append(df)
    i_counter+=1
    if i_counter%10==1:
        print(i_counter,datetime.datetime.now())
df_shopper_by_month=pd.concat(shopper_by_day_list)
print(df_shopper_by_month.shape)

del shopper_by_day_list
df_shopper_by_month['shop_month']=df_shopper_by_month['transaction_dt'].apply(lambda x: x[:7])
df_shopper_by_month=df_shopper_by_month.groupby(['customer_id_hashed','shop_month'])['sales','trans'].sum().reset_index()

print(df_shopper_by_month.shape)

gc.collect()
print(datetime.datetime.now())


1 2019-12-05 11:54:21.987524
11 2019-12-05 12:01:46.851708
21 2019-12-05 12:13:25.206225
31 2019-12-05 12:20:42.437406
41 2019-12-05 12:28:45.574772
51 2019-12-05 12:37:10.856941
61 2019-12-05 12:45:37.354519
(16571933, 4)
(11969887, 4)
2019-12-05 12:52:32.269376


In [14]:
df_shopper_by_day_class=pd.concat(shopper_by_day_department_list)
df_shopper_by_day_class=pd.merge(df_shopper_by_day_class,mapping_division_class,on="class_code_id",how="left")
gc.collect()

print(datetime.datetime.now())

df_shopper_by_day_class.head(3)

2019-12-05 12:53:29.115748


,customer_id_hashed,transaction_dt,class_code_id,sales,division_id,division_desc
0,00002f57890442353de21f2ab1b17556089a69ad211c66...,2018-09-01,35039,20.0,4,SOFTHOME
1,00002f57890442353de21f2ab1b17556089a69ad211c66...,2018-09-01,35045,35.0,4,SOFTHOME
2,0003ca38cdd0e0b46b6971904e5ded3d313705e7b3cb7c...,2018-09-01,11001,5.0,1,FOOD


In [15]:
df_shopper_by_day_division=df_shopper_by_day_class.groupby(["customer_id_hashed","transaction_dt","division_id","division_desc"])['sales'].sum().to_frame().reset_index()
df_shopper_by_day_division.head(2)

,customer_id_hashed,transaction_dt,division_id,division_desc,sales
0,000001dadc0265bf9d250566d74e0006323f18b5826641...,2018-09-22,1,FOOD,1.6
1,000001dadc0265bf9d250566d74e0006323f18b5826641...,2018-09-22,10,DIVISION 10,0.1


In [16]:
del df_shopper_by_day_class
gc.collect()

0

In [17]:
df_shopper_by_day_division.head(2)

,customer_id_hashed,transaction_dt,division_id,division_desc,sales
0,000001dadc0265bf9d250566d74e0006323f18b5826641...,2018-09-22,1,FOOD,1.6
1,000001dadc0265bf9d250566d74e0006323f18b5826641...,2018-09-22,10,DIVISION 10,0.1


In [18]:
df_shopper_by_day_division.shape,df_shopper_by_day_division['customer_id_hashed'].nunique()

((35243934, 5), 6007038)

In [19]:
# Unscribed ids & emails

ids_unscribed=pd.read_csv("/home/jian/BigLots/unsubscribe/Unsub_JH_Final20191105Hashed.zip",
                          dtype=str,compression="zip",usecols=["hashed_customer_num"])
ids_unscribed=ids_unscribed.rename(columns={"hashed_customer_num":"customer_id_hashed"})
ids_unscribed['subscription']="Unsubscribed"
ids_unscribed=ids_unscribed.drop_duplicates()
print(ids_unscribed.shape,ids_unscribed['customer_id_hashed'].nunique())

(2771375, 2) 2771375


In [20]:
df_shopper_by_day_division=pd.merge(df_shopper_by_day_division,ids_unscribed,on="customer_id_hashed",how="left")
df_shopper_by_day_division['subscription']=df_shopper_by_day_division['subscription'].fillna("Default_Subscribe")

In [21]:
df_shopper_by_day_division.head(5)

,customer_id_hashed,transaction_dt,division_id,division_desc,sales,subscription
0,000001dadc0265bf9d250566d74e0006323f18b5826641...,2018-09-22,1,FOOD,1.6,Default_Subscribe
1,000001dadc0265bf9d250566d74e0006323f18b5826641...,2018-09-22,10,DIVISION 10,0.1,Default_Subscribe
2,000001dadc0265bf9d250566d74e0006323f18b5826641...,2018-09-22,2,CONSUMABLES,6.0,Default_Subscribe
3,000001dadc0265bf9d250566d74e0006323f18b5826641...,2018-09-22,3,SEASONAL,45.0,Default_Subscribe
4,000001dadc0265bf9d250566d74e0006323f18b5826641...,2018-09-22,4,SOFTHOME,6.0,Default_Subscribe


# 1. Id count

In [22]:
df_shopper_by_month['shopping_behavior']=1

In [23]:
def month_add_interval(str_YYYY_mm,int_m_interval):
    YYYY_int=int(str_YYYY_mm.split("-")[0])
    mm_int=int(str_YYYY_mm.split("-")[1])
    
    mm_int_new=mm_int+int_m_interval
    if mm_int_new>=13:
        YYYY_int+=1
        mm_int_new=mm_int_new-12
    return str(YYYY_int)+"-"+str(mm_int_new).zfill(2)

In [24]:
df_output_long.head(2)

,customer_id_hashed,sign_up_date,registerd_month
201365,14d2626ab97b1895f5c0955d635f2d859d6b7773d24a18...,2019-11-30,2019-11
1264,0d486acc9776c5daad35c2bde37be6d25e59ae04681585...,2019-11-30,2019-11


In [25]:
df_output_1_count=df_output_long.copy()
for i in range(19):
    df_output_1_count["M"+str(i)]=df_output_1_count['registerd_month'].apply(lambda x: month_add_interval(x,i))
    df_output_1_count=pd.merge(df_output_1_count,df_shopper_by_month[['customer_id_hashed','shop_month','shopping_behavior']],
                            left_on=['customer_id_hashed',"M"+str(i)],
                            right_on=['customer_id_hashed',"shop_month"],how="left")
    del df_output_1_count['shop_month']
    del df_output_1_count["M"+str(i)]
    df_output_1_count['shopping_behavior']=df_output_1_count['shopping_behavior'].fillna(0)
    df_output_1_count=df_output_1_count.rename(columns={"shopping_behavior":"M"+str(i)})
    print(i,datetime.datetime.now())
    
print(df_output_1_count.shape)

0 2019-12-05 12:59:25.594695
1 2019-12-05 13:00:36.435969
2 2019-12-05 13:01:48.879446
3 2019-12-05 13:03:02.293284
4 2019-12-05 13:04:16.199189
5 2019-12-05 13:05:30.620173
6 2019-12-05 13:06:47.171883
7 2019-12-05 13:08:05.115019
8 2019-12-05 13:09:21.703393
9 2019-12-05 13:10:39.330594
10 2019-12-05 13:11:57.842213
11 2019-12-05 13:13:18.905667
12 2019-12-05 13:14:41.019678
13 2019-12-05 13:16:02.848988
14 2019-12-05 13:17:25.376400
15 2019-12-05 13:18:49.582632
16 2019-12-05 13:20:14.976329
17 2019-12-05 13:21:40.892588
18 2019-12-05 13:23:07.729248
(8763575, 22)


In [26]:
df_id_count=df_output_1_count.groupby("registerd_month")['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"id_new_signed_count"})
df_id_shopper_count_sum=df_output_1_count.groupby(['registerd_month'])[[x for x in df_output_1_count.columns.tolist() if x[0]=="M"]].sum().reset_index()

df_output_1_count=pd.merge(df_id_count,df_id_shopper_count_sum,on="registerd_month",how="outer")

In [27]:
df_output_1_count

,registerd_month,id_new_signed_count,M0,M1,M2,M3,M4,M5,M6,M7,...,M9,M10,M11,M12,M13,M14,M15,M16,M17,M18
0,2018-09,559549,390823.0,90361.0,87480.0,89580.0,59316.0,53315.0,59465.0,57168.0,...,52844.0,54849.0,53022.0,54257.0,53007.0,61888.0,0.0,0.0,0.0,0.0
1,2018-10,441767,261677.0,73787.0,67800.0,42327.0,38221.0,41935.0,39953.0,38174.0,...,36938.0,35403.0,36248.0,38088.0,43756.0,0.0,0.0,0.0,0.0,0.0
2,2018-11,824107,466209.0,136546.0,74056.0,63221.0,69674.0,65457.0,62501.0,58977.0,...,57444.0,59262.0,62430.0,86113.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-12,843469,484238.0,78858.0,60452.0,64319.0,60109.0,56290.0,54078.0,56224.0,...,51853.0,54453.0,72466.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-01,554172,357651.0,79372.0,71564.0,62632.0,57048.0,53218.0,54413.0,50388.0,...,49653.0,57691.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2019-02,495334,317587.0,78865.0,57211.0,50433.0,45844.0,46079.0,43145.0,41748.0,...,48797.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2019-03,551970,373413.0,84743.0,62312.0,54405.0,53883.0,49178.0,47412.0,47451.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2019-04,529464,352694.0,78352.0,59453.0,56434.0,49932.0,48044.0,47322.0,55274.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2019-05,518120,355840.0,77480.0,61986.0,53535.0,49193.0,48066.0,54817.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2019-06,515025,360205.0,85111.0,60986.0,53499.0,51413.0,57516.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 2. Sales

In [28]:
df_output_2_sales=df_output_long.copy()
for i in range(19):
    df_output_2_sales["M"+str(i)]=df_output_2_sales['registerd_month'].apply(lambda x: month_add_interval(x,i))
    df_output_2_sales=pd.merge(df_output_2_sales,df_shopper_by_month[['customer_id_hashed','shop_month','sales']],
                            left_on=['customer_id_hashed',"M"+str(i)],
                            right_on=['customer_id_hashed',"shop_month"],how="left")
    del df_output_2_sales['shop_month']
    del df_output_2_sales["M"+str(i)]
    df_output_2_sales['sales']=df_output_2_sales['sales'].fillna(0)
    df_output_2_sales=df_output_2_sales.rename(columns={"sales":"M"+str(i)})
    print(i,datetime.datetime.now())
print(df_output_2_sales.shape)


0 2019-12-05 13:24:28.150723
1 2019-12-05 13:25:39.348474
2 2019-12-05 13:26:52.703869
3 2019-12-05 13:28:07.595983
4 2019-12-05 13:29:28.280595
5 2019-12-05 13:30:45.123275
6 2019-12-05 13:32:09.971371
7 2019-12-05 13:33:37.689945
8 2019-12-05 13:35:03.642278
9 2019-12-05 13:36:32.274143
10 2019-12-05 13:37:54.956015
11 2019-12-05 13:39:28.254187
12 2019-12-05 13:40:54.504972
13 2019-12-05 13:42:28.668089
14 2019-12-05 13:43:55.674862
15 2019-12-05 13:45:32.075131
16 2019-12-05 13:47:10.728855
17 2019-12-05 13:48:50.792629
18 2019-12-05 13:50:22.470473
(8763575, 22)


In [29]:
df_id_shopper_sales_sum=df_output_2_sales.groupby(['registerd_month'])[[x for x in df_output_2_sales.columns.tolist() if x[0]=="M"]].sum().reset_index()

df_output_2_sales=pd.merge(df_id_count,df_id_shopper_sales_sum,on="registerd_month",how="outer")

# 3. Trans

In [30]:
df_output_3_trans=df_output_long.copy()
for i in range(19):
    df_output_3_trans["M"+str(i)]=df_output_3_trans['registerd_month'].apply(lambda x: month_add_interval(x,i))
    df_output_3_trans=pd.merge(df_output_3_trans,df_shopper_by_month[['customer_id_hashed','shop_month','trans']],
                            left_on=['customer_id_hashed',"M"+str(i)],
                            right_on=['customer_id_hashed',"shop_month"],how="left")
    del df_output_3_trans['shop_month']
    del df_output_3_trans["M"+str(i)]
    df_output_3_trans['trans']=df_output_3_trans['trans'].fillna(0)
    df_output_3_trans=df_output_3_trans.rename(columns={"trans":"M"+str(i)})
    print(i,datetime.datetime.now())
print(df_output_3_trans.shape)


0 2019-12-05 13:51:48.424886
1 2019-12-05 13:53:08.089434
2 2019-12-05 13:54:28.078751
3 2019-12-05 13:55:50.535081
4 2019-12-05 13:57:14.126142
5 2019-12-05 13:58:29.720256
6 2019-12-05 13:59:56.295394
7 2019-12-05 14:01:19.254366
8 2019-12-05 14:02:47.177105
9 2019-12-05 14:04:15.682690
10 2019-12-05 14:05:46.016266
11 2019-12-05 14:07:17.728720
12 2019-12-05 14:08:42.961057
13 2019-12-05 14:10:12.607600
14 2019-12-05 14:11:44.987598
15 2019-12-05 14:13:23.578166
16 2019-12-05 14:15:02.167184
17 2019-12-05 14:16:41.983152
18 2019-12-05 14:18:14.773568
(8763575, 22)


In [31]:
df_id_shopper_trans_sum=df_output_3_trans.groupby(['registerd_month'])[[x for x in df_output_3_trans.columns.tolist() if x[0]=="M"]].sum().reset_index()

df_output_3_trans=pd.merge(df_id_count,df_id_shopper_trans_sum,on="registerd_month",how="outer")

In [32]:
writer=pd.ExcelWriter(writer_folder+"BL_LTV_retention_rate_for_new_sign_ups_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output_1_count.to_excel(writer,"table_1_count",index=False)
df_output_2_sales.to_excel(writer,"table_2_sales",index=False)
df_output_3_trans.to_excel(writer,"table_3_trans",index=False)

writer.save()

# Sales by Division table

In [33]:
df_shopper_by_day_division['transaction_dt']=pd.to_datetime(df_shopper_by_day_division['transaction_dt'],format="%Y-%m-%d").dt.date
df_output_long['sign_up_date']=pd.to_datetime(df_output_long['sign_up_date'],format="%Y-%m-%d").dt.date
df_output_long=pd.merge(df_output_long,df_shopper_by_day_division,on="customer_id_hashed",how="left")

In [34]:
df_output_long['day_diff']=(df_output_long['transaction_dt']-df_output_long['sign_up_date']).dt.days
df_output_long['month_diff']=np.floor(df_output_long['day_diff']/30)

df_output_long.head(2)

,customer_id_hashed,sign_up_date,registerd_month,transaction_dt,division_id,division_desc,sales,subscription,day_diff,month_diff
0,14d2626ab97b1895f5c0955d635f2d859d6b7773d24a18...,2019-11-30,2019-11,2019-11-30,1,FOOD,3.0,Default_Subscribe,0.0,0.0
1,14d2626ab97b1895f5c0955d635f2d859d6b7773d24a18...,2019-11-30,2019-11,2019-11-30,2,CONSUMABLES,20.6,Default_Subscribe,0.0,0.0


In [35]:
df_output_long[df_output_long['month_diff']<0].shape

(55300, 10)

In [36]:
df_output_long.shape

(38000471, 10)

In [37]:
df_output_long[df_output_long['month_diff']<0]['customer_id_hashed'].nunique()

22253

In [38]:
df_output_long['customer_id_hashed'].nunique()

8763575

In [39]:
df_output_long['month_diff_str']=df_output_long['month_diff'].apply(lambda x: "M_"+str(x).split(".")[0].zfill(2))

In [40]:
df_output_pivot_0=df_output_long.groupby(['registerd_month','division_id','division_desc','month_diff_str'])['sales'].sum().to_frame().reset_index()
df_output_pivot_0=df_output_pivot_0.pivot_table(index=['registerd_month','division_id','division_desc'],columns="month_diff_str",values="sales").reset_index()

In [41]:
df_output_pivot_0['division_id']=df_output_pivot_0['division_id'].astype(int)
df_output_pivot_0=df_output_pivot_0.sort_values(["registerd_month",'division_id'])

In [42]:
df_output_pivot_1=df_output_pivot_0[[x for x  in df_output_pivot_0.columns.tolist() if "-" not in x]]
df_output_pivot_2=df_output_pivot_1[df_output_pivot_1['division_desc']!="DIVISION 8 (EMP DISCOUNT)"]
df_output_pivot_2=df_output_pivot_2[df_output_pivot_2['division_desc']!="DIVISION 10"]


In [43]:
writer=pd.ExcelWriter(writer_folder+"BL_LTV_by_division_rolling_months_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output_pivot_2.to_excel(writer,"table_final",index=False)
df_output_pivot_1.to_excel(writer,"table_exclude_outlier",index=False)
df_output_pivot_0.to_excel(writer,"table_original",index=False)
writer.save()

# By email subscription

In [44]:
df_new_signs_since_Sep18=pd.DataFrame()

for file in list_weekly_signs_files:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','sign_up_date'])
    df=df[df['sign_up_date']>=track_start_date]
    df_new_signs_since_Sep18=df_new_signs_since_Sep18.append(df)
    
dfdetail_1=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/All Rewards Members 2018-02-04 - 2019-05-04.zip",
                         dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','sign_up_date'])
dfdetail_2=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/MediaStorm Rewards Master P4 2019 - no transaction info.zip",
                         dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','sign_up_date'])

dfdetail_1=dfdetail_1[dfdetail_1['sign_up_date']>=track_start_date]
dfdetail_2=dfdetail_2[dfdetail_2['sign_up_date']>=track_start_date]

df_new_signs_since_Sep18=df_new_signs_since_Sep18.append(dfdetail_1).append(dfdetail_2)

del dfdetail_1
del dfdetail_2

gc.collect()

print(df_new_signs_since_Sep18.shape)
df_new_signs_since_Sep18=df_new_signs_since_Sep18.sort_values("sign_up_date",ascending=False).drop_duplicates("customer_id_hashed")
print(df_new_signs_since_Sep18.shape)

month_date_count=df_new_signs_since_Sep18[['sign_up_date']].drop_duplicates()
month_date_count['registerd_month']=month_date_count['sign_up_date'].apply(lambda x: x[:7])
month_date_count=month_date_count.groupby("registerd_month")['sign_up_date'].nunique().to_frame().reset_index()

df_output_long=df_new_signs_since_Sep18[['customer_id_hashed','sign_up_date']]
df_output_long['registerd_month']=df_output_long['sign_up_date'].apply(lambda x: x[:7])
del df_new_signs_since_Sep18

list_ids_new_signs_since_Sep18=df_output_long['customer_id_hashed'].tolist()
gc.collect()

print(df_output_long.shape)
df_output_long.head(3)

(12142496, 3)
(8763575, 3)
(8763575, 3)


,customer_id_hashed,sign_up_date,registerd_month
201365,14d2626ab97b1895f5c0955d635f2d859d6b7773d24a18...,2019-11-30,2019-11
1264,0d486acc9776c5daad35c2bde37be6d25e59ae04681585...,2019-11-30,2019-11
1267,ac101138924bb47feb91fb5ce10178c509ac5830bc9a87...,2019-11-30,2019-11


In [45]:
df_output_long=pd.merge(df_output_long,ids_unscribed,on="customer_id_hashed",how="left")
df_output_long['subscription']=df_output_long['subscription'].fillna("Default_Subscribe")

In [46]:
# 1 id_count
df_output_long.head(2)

,customer_id_hashed,sign_up_date,registerd_month,subscription
0,14d2626ab97b1895f5c0955d635f2d859d6b7773d24a18...,2019-11-30,2019-11,Default_Subscribe
1,0d486acc9776c5daad35c2bde37be6d25e59ae04681585...,2019-11-30,2019-11,Default_Subscribe


In [47]:
df_output_1_count_subscription=df_output_long.copy()
for i in range(19):
    df_output_1_count_subscription["M"+str(i)]=df_output_1_count_subscription['registerd_month'].apply(lambda x: month_add_interval(x,i))
    df_output_1_count_subscription=pd.merge(df_output_1_count_subscription,df_shopper_by_month[['customer_id_hashed','shop_month','shopping_behavior']],
                            left_on=['customer_id_hashed',"M"+str(i)],
                            right_on=['customer_id_hashed',"shop_month"],how="left")
    del df_output_1_count_subscription['shop_month']
    del df_output_1_count_subscription["M"+str(i)]
    df_output_1_count_subscription['shopping_behavior']=df_output_1_count_subscription['shopping_behavior'].fillna(0)
    df_output_1_count_subscription=df_output_1_count_subscription.rename(columns={"shopping_behavior":"M"+str(i)})
    print(i,datetime.datetime.now())
    
print(df_output_1_count_subscription.shape)

0 2019-12-05 14:30:40.909538
1 2019-12-05 14:32:01.320264
2 2019-12-05 14:33:16.865205
3 2019-12-05 14:34:36.977049
4 2019-12-05 14:35:54.299777
5 2019-12-05 14:37:17.598116
6 2019-12-05 14:38:36.875505
7 2019-12-05 14:40:04.622098
8 2019-12-05 14:41:25.901255
9 2019-12-05 14:42:55.520861
10 2019-12-05 14:44:25.798839
11 2019-12-05 14:45:59.411694
12 2019-12-05 14:47:33.737893
13 2019-12-05 14:49:01.989960
14 2019-12-05 14:50:30.035510
15 2019-12-05 14:52:09.247734
16 2019-12-05 14:53:41.791134
17 2019-12-05 14:55:23.399053
18 2019-12-05 14:57:05.952893
(8763575, 23)


In [48]:
df_output_1_count_subscription.head(5)

,customer_id_hashed,sign_up_date,registerd_month,subscription,M0,M1,M2,M3,M4,M5,...,M9,M10,M11,M12,M13,M14,M15,M16,M17,M18
0,14d2626ab97b1895f5c0955d635f2d859d6b7773d24a18...,2019-11-30,2019-11,Default_Subscribe,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0d486acc9776c5daad35c2bde37be6d25e59ae04681585...,2019-11-30,2019-11,Default_Subscribe,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ac101138924bb47feb91fb5ce10178c509ac5830bc9a87...,2019-11-30,2019-11,Default_Subscribe,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,119998cf5aefdf2f58451ed1360b37b23043556f00a7d0...,2019-11-30,2019-11,Default_Subscribe,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,be9eae9bb8c41b04912a6bf94f0331455b2830fe356d80...,2019-11-30,2019-11,Default_Subscribe,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
df_id_count_subscription=df_output_1_count_subscription.groupby(["registerd_month","subscription"])['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"id_new_signed_count"})
df_id_shopper_count_sum_subscription=df_output_1_count_subscription.groupby(["registerd_month","subscription"])[[x for x in df_output_1_count_subscription.columns.tolist() if x[0]=="M"]].sum().reset_index()

df_output_1_count_subscription=pd.merge(df_id_count_subscription,df_id_shopper_count_sum_subscription,on=["registerd_month","subscription"],how="outer")

In [50]:
# 2

df_output_2_sales_subscription=df_output_long.copy()
for i in range(19):
    df_output_2_sales_subscription["M"+str(i)]=df_output_2_sales_subscription['registerd_month'].apply(lambda x: month_add_interval(x,i))
    df_output_2_sales_subscription=pd.merge(df_output_2_sales_subscription,df_shopper_by_month[['customer_id_hashed','shop_month','sales']],
                            left_on=['customer_id_hashed',"M"+str(i)],
                            right_on=['customer_id_hashed',"shop_month"],how="left")
    del df_output_2_sales_subscription['shop_month']
    del df_output_2_sales_subscription["M"+str(i)]
    df_output_2_sales_subscription['sales']=df_output_2_sales_subscription['sales'].fillna(0)
    df_output_2_sales_subscription=df_output_2_sales_subscription.rename(columns={"sales":"M"+str(i)})
    print(i,datetime.datetime.now())
print(df_output_2_sales_subscription.shape)


0 2019-12-05 14:58:33.123300
1 2019-12-05 14:59:48.790536
2 2019-12-05 15:01:03.622551
3 2019-12-05 15:02:25.518835
4 2019-12-05 15:03:49.627905
5 2019-12-05 15:05:14.275015
6 2019-12-05 15:06:42.446764
7 2019-12-05 15:08:10.352194
8 2019-12-05 15:09:31.154437
9 2019-12-05 15:10:51.609507
10 2019-12-05 15:12:23.299604
11 2019-12-05 15:13:58.020668
12 2019-12-05 15:15:33.025455
13 2019-12-05 15:17:09.003827
14 2019-12-05 15:18:38.227332
15 2019-12-05 15:20:13.170661
16 2019-12-05 15:21:49.677236
17 2019-12-05 15:23:30.880611
18 2019-12-05 15:25:14.505610
(8763575, 23)


In [51]:
df_id_shopper_sales_sum_subscription=df_output_2_sales_subscription.groupby(['registerd_month','subscription'])[[x for x in df_output_2_sales_subscription.columns.tolist() if x[0]=="M"]].sum().reset_index()

df_output_2_sales_subscription=pd.merge(df_id_count_subscription,df_id_shopper_sales_sum_subscription,on=["registerd_month","subscription"],how="outer")


In [52]:
# 3

df_output_3_trans_subscription=df_output_long.copy()
for i in range(19):
    df_output_3_trans_subscription["M"+str(i)]=df_output_3_trans_subscription['registerd_month'].apply(lambda x: month_add_interval(x,i))
    df_output_3_trans_subscription=pd.merge(df_output_3_trans_subscription,df_shopper_by_month[['customer_id_hashed','shop_month','trans']],
                            left_on=['customer_id_hashed',"M"+str(i)],
                            right_on=['customer_id_hashed',"shop_month"],how="left")
    del df_output_3_trans_subscription['shop_month']
    del df_output_3_trans_subscription["M"+str(i)]
    df_output_3_trans_subscription['trans']=df_output_3_trans_subscription['trans'].fillna(0)
    df_output_3_trans_subscription=df_output_3_trans_subscription.rename(columns={"trans":"M"+str(i)})
    print(i,datetime.datetime.now())
print(df_output_3_trans_subscription.shape)


0 2019-12-05 15:26:44.342166
1 2019-12-05 15:28:04.856249
2 2019-12-05 15:29:21.785951
3 2019-12-05 15:30:38.313376
4 2019-12-05 15:32:00.753980
5 2019-12-05 15:33:27.004347
6 2019-12-05 15:34:55.776728
7 2019-12-05 15:36:26.188997
8 2019-12-05 15:37:57.082904
9 2019-12-05 15:39:20.577829
10 2019-12-05 15:40:44.587571
11 2019-12-05 15:42:11.451906
12 2019-12-05 15:43:47.705670
13 2019-12-05 15:45:24.363014
14 2019-12-05 15:47:00.701967
15 2019-12-05 15:48:32.976055
16 2019-12-05 15:50:05.052723
17 2019-12-05 15:51:40.840759
18 2019-12-05 15:53:29.230388
(8763575, 23)


In [53]:
df_id_shopper_trans_sum_subscription=df_output_3_trans_subscription.groupby(['registerd_month','subscription'])[[x for x in df_output_3_trans_subscription.columns.tolist() if x[0]=="M"]].sum().reset_index()

df_output_3_trans_subscription=pd.merge(df_id_count_subscription,df_id_shopper_trans_sum_subscription,on=["registerd_month",'subscription'],how="outer")

In [54]:
writer=pd.ExcelWriter(writer_folder+"BL_LTV_retention_rate_for_new_sign_ups_subscribetion_label_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output_1_count_subscription.to_excel(writer,"table_1_count",index=False)
df_output_2_sales_subscription.to_excel(writer,"table_2_sales",index=False)
df_output_3_trans_subscription.to_excel(writer,"table_3_trans",index=False)

writer.save()